# 影響関数（influence function）


- [Koh & Liang (2017)](http://proceedings.mlr.press/v70/koh17a/koh17a.pdf)が提案
- [Hara et al. (2019)](https://proceedings.neurips.cc/paper/2019/file/5f14615696649541a025d3d0f8e0447f-Paper.pdf)はconvexでない損失関数にも使えるよう拡張


## LOO

1つのインスタンスを抜いた状態で訓練した場合にどれだけ予測値が変わるかを考える

n回学習し直す必要があるので現実的ではない


## Influence Function

近似


## FastFI

更に近似

## 参考文献

- [Influence Functionでインスタンスの重要度を解釈する - Dropout](https://dropout009.hatenablog.com/entry/2021/07/19/223929)